preprocessing pipilne by syjjsybrenda98@gmail.com

smchou가 만든 raw nifti 파일에서

복부까지 보이는 파일이 있으니까, voxel spacing * shape을 해서 200/voxel_spacing[2] 갯수만큼 까지만 새 디렉토리에 저장

0 이하, 100 이상을 0으로 clip하고, 0 이상인 값들에 대해서 clustering.

제일 큰 cluster (mask) 만 남기고 0으로 suppression.

mask를 fill hole 하기

만들어진 mask 부분만 MRI에서 떼어내기

혹시 뇌 안쪽에 있던 음수값을 0으로 바꾸기

양수 범위만 box로 crop하기

~~저장된 파일을 가지고 더 refine할 방안...~~

~~array로 뽑았을 때, 0번째 slice가 몸통쪽이고 마지막 slice가 머리 꼭대기 부분이다.~~

~~몸통쪽 (slice index 작은쪽)부터 한 슬라이스씩 추가로 없애서 tmp_nifti에 담고 (없앤 nifty는 다른 tmp_nifti2에 저장) clustering을 시행했을 때~~

~~cluster가 두개 이상이 되는 순간에 슬라이스씩 나누는 걸 멈추고 가장 큰 cluster만 남김.~~

~~그리고 tmp_nifti2와 z축 방향으로 합치기~~

subdural hematoma 를 mri_synthstrip이 잘라낼 수 있으므로, 튀는 값이 없게, ct값의 median 값으로 전체 voxel을 upper clip 한 다음에 mri_synthstrip하니까 깔끔하게 bet됨

<!-- 마지막으로 voxel spacing 맞추기. ICH_brm 에서 deep bleed를 이용해서 pre대비 post가 30% 이상 증가했을 경우 HE인 것으로 labeling을 해야하기 때문 -->
ICH_brm 에서 deep bleed를 이용해서 pre대비 post가 30% 이상 증가했을 경우 HE인 것으로 labeling을 해야하는데,
segmentation mask를 구해낸 다음에, 원래 CT의 header의 voxel spacing 이용해서 nifti로 저장하면 fslmath -v로 실제 volume이 얼마나 증가했는 지 알 수 있음

In [1]:
import os
import monai
import nibabel as nib
import numpy as np
import wandb
from tqdm import tqdm
# from fsl.data.image import Image
# from fsl.wrappers import fslmaths, cluster
import subprocess
# pip install moviepy imageio

In [2]:
pwd

'/ssd_data1/syjeong/hematoma/server202/preprocess'

In [35]:
pwd

'/ssd_data1/syjeong/hematoma/wandb'

In [3]:
dest = "../data/cut20cm"
source = "../data/raw/nifti"

In [19]:
os.makedirs(dest, exist_ok=True)
# brainct_final: internal, 
# ICH_brm: external/pre (_1.nii.gz), /post(_2.nii.gz)
os.makedirs(os.path.join(dest, 'internal', 'pre'), exist_ok=True)
os.makedirs(os.path.join(dest, 'internal', 'post'), exist_ok=True)
os.makedirs(os.path.join(dest, 'external', 'pre'), exist_ok=True)
os.makedirs(os.path.join(dest, 'external', 'post'), exist_ok=True)

In [13]:
os.chdir("../wandb")

# 1. 복부까지 보이는 파일이 있으니까, voxel spacing * shape을 해서 200/voxel_spacing[2] 갯수만큼 까지만 새 디렉토리에 저장

In [ ]:
cutting = 200 # mm

## brainct_final

In [23]:
dataset = "brainct_final"
for data in os.listdir(os.path.join(source, dataset)):
    img = nib.load(os.path.join(source, dataset, data))
    vs = img.header.get_zooms()
    shape = img.shape
    # mm = np.array(shape) * np.array(vs)
    cut_slice = int(np.ceil(cutting/vs[2]))
    new = img.get_fdata()[:, :, -min(cut_slice, shape[2]):]
    new = nib.Nifti1Image(new, img.affine, img.header)
    nib.save(new, os.path.join(dest, 'internal/pre', data))

## ICH_brm

In [25]:
dataset="ICH_brm/indir"
for data in os.listdir(os.path.join(source, dataset)):
    if '_1.nii.gz' in data:
        subdir = 'pre'
    elif '_2.nii.gz' in data:
        subdir = 'post'
    else:
        print(data)
        continue
    img = nib.load(os.path.join(source, dataset, data))
    vs = img.header.get_zooms()
    shape = img.shape
    # mm = np.array(shape) * np.array(vs)
    cut_slice = int(np.ceil(cutting/vs[2]))
    new = img.get_fdata()[:, :, -min(cut_slice, shape[2]):]
    new = nib.Nifti1Image(new, img.affine, img.header)
    nib.save(new, os.path.join(dest, 'external', subdir, data))

# 2. 0 이하, 100 이상을 0으로 clip하고, 0 이상인 값들에 대해서 clustering.

In [20]:
os.makedirs('../data/clipNClustered/internal/pre', exist_ok=True)
os.makedirs('../data/clipNClustered/internal/post', exist_ok=True)
os.makedirs('../data/clipNClustered/external/pre', exist_ok=True)
os.makedirs('../data/clipNClustered/external/post', exist_ok=True)

In [21]:
os.makedirs('../data/clusterMask/internal/pre', exist_ok=True)
os.makedirs('../data/clusterMask/internal/post', exist_ok=True)
os.makedirs('../data/clusterMask/external/pre', exist_ok=True)
os.makedirs('../data/clusterMask/external/post', exist_ok=True)

In [26]:
with wandb.init(project="HE", name="brinct_final", config={"data":"brinct_final", "directory":"clipNClustered internal pre"}) as run:
   
    for data in tqdm(sorted(os.listdir(os.path.join(dest, 'internal/pre')))): # brainct_final
        img = nib.load(os.path.join(dest, 'internal/pre', data))
        array = img.get_fdata()
        array = np.clip(array, 0, 100)
        array = np.where(array==100, 0, array)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        print(data, img.shape)
        file_name = os.path.join('../data/clusterMask/internal/pre', data)
        nib.save(img, file_name)
        index = subprocess.run(['cluster', f'--in={file_name}', '--thresh=0.1', f'--oindex={file_name}', f'--minclustersize={512*512}'], capture_output=True).stdout
        index = int(index.decode().split('\n')[1].split('\t')[0])
        img = nib.load(file_name)
        array = img.get_fdata()
        array = np.where(array==index, 1, 0)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, file_name)
        subprocess.run(['fslmaths', f'{file_name}', '-fillh', f'{file_name}']) # 100 이상 voxel을 가졌던 뇌 안쪽 부분 채우기
        img = nib.load(os.path.join(dest, 'internal/pre', data))
        array = img.get_fdata()
        mask = nib.load(file_name)
        mask = mask.get_fdata()
        array[mask==0] = 0
        array = np.where(array<=0, 0, array) # 혹시 뇌 안쪽에 음수값이 있었을 경우에 제거.
        index = np.argwhere(array>0)
        array = array[np.min(index[:, 0]):np.max(index[:, 0]), np.min(index[:, 1]):np.max(index[:, 1]), np.min(index[:, 2]):np.max(index[:, 2])]
        print(array.shape)# H W T
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, os.path.join('../data/clipNClustered/internal/pre', data))
        array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
        array = np.transpose(array, (2, 3, 0, 1)) #  Video must be atleast 4 dimensions: time, channels, height, width
        # wandb.log({f'clipNClustered/internal/{data}': wandb.Video(array, fps=4), f'clipNClustered/internal/{data}/scaled': wandb.Video(np.clip(array, 0, 80), fps=4)}) 
        wandb.log({f'clipNClustered/internal/pre/{data}': wandb.Video(array, fps=4)}) 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: syjjsybrenda98 (seongyunjeong). Use `wandb login --relogin` to force relogin


  0%|          | 0/568 [00:00<?, ?it/s]

07701739.nii.gz (512, 512, 36)
(387, 443, 32)


  0%|          | 1/568 [00:13<2:03:22, 13.06s/it]

07751950.nii.gz (512, 512, 38)
(349, 462, 35)


...

100%|█████████▉| 566/568 [1:58:47<00:19,  9.60s/it]

71650003.nii.gz (512, 512, 35)
(372, 388, 31)


100%|█████████▉| 567/568 [1:58:55<00:08,  8.95s/it]

72605291.nii.gz (512, 512, 51)
(412, 437, 50)


100%|██████████| 568/568 [1:59:04<00:00, 12.58s/it]


In [27]:
with wandb.init(project="HE", name="ICH_brm _1", config={"data":"ICH_brm _1", "directory":"clipNClustered external pre"}) as run:
    
    for data in tqdm(sorted(os.listdir(os.path.join(dest, 'external/pre')))): # ICH_brm _1
        img = nib.load(os.path.join(dest, 'external/pre', data))
        array = img.get_fdata()
        array = np.clip(array, 0, 100)
        array = np.where(array==100, 0, array)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        print(data, img.shape)
        file_name = os.path.join('../data/clusterMask/external/pre', data)
        nib.save(img, file_name)
        index = subprocess.run(['cluster', f'--in={file_name}', '--thresh=0.1', f'--oindex={file_name}', f'--minclustersize={512*512}'], capture_output=True).stdout
        index = int(index.decode().split('\n')[1].split('\t')[0])
        img = nib.load(file_name)
        array = img.get_fdata()
        array = np.where(array==index, 1, 0)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, file_name)
        subprocess.run(['fslmaths', f'{file_name}', '-fillh', f'{file_name}']) # 100 이상 voxel을 가졌던 뇌 안쪽 부분 채우기
        img = nib.load(os.path.join(dest, 'external/pre', data))
        array = img.get_fdata()
        mask = nib.load(file_name)
        mask = mask.get_fdata()
        array[mask==0] = 0
        array = np.where(array<=0, 0, array) # 혹시 뇌 안쪽에 음수값이 있었을 경우에 제거.
        index = np.argwhere(array>0)
        array = array[np.min(index[:, 0]):np.max(index[:, 0]), np.min(index[:, 1]):np.max(index[:, 1]), np.min(index[:, 2]):np.max(index[:, 2])]
        print(array.shape)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, os.path.join('../data/clipNClustered/external/pre', data))
        array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
        array = np.transpose(array, (2, 3, 0, 1)) #  Video must be atleast 4 dimensions: time, channels, height, width
        # wandb.log({f'clipNClustered/internal/{data}': wandb.Video(array, fps=4), f'clipNClustered/internal/{data}/scaled': wandb.Video(np.clip(array, 0, 80), fps=4)}) 
        wandb.log({f'clipNClustered/external/pre/{data}': wandb.Video(array, fps=4)}) 

  0%|          | 0/309 [00:00<?, ?it/s]

00037018_1.nii.gz (512, 569, 30)
(458, 442, 26)


  0%|          | 1/309 [00:05<30:33,  5.95s/it]

00038043_1.nii.gz (512, 542, 30)
(416, 467, 25)


  1%|          | 2/309 [00:11<30:30,  5.96s/it]

...

 99%|█████████▉| 307/309 [31:12<00:09,  4.78s/it]

01891080_1.nii.gz (512, 602, 34)
(499, 499, 28)


100%|█████████▉| 308/309 [31:17<00:04,  4.86s/it]

01893382_1.nii.gz (510, 510, 31)
(447, 472, 28)


100%|██████████| 309/309 [31:21<00:00,  6.09s/it]


In [28]:
with wandb.init(project="HE", name="ICH_brm _2", config={"data":"ICH_brm _2", "directory":"clipNClustered external post"}) as run:
    
    for data in tqdm(sorted(os.listdir(os.path.join(dest, 'external/post')))): # ICH_brm _2
        img = nib.load(os.path.join(dest, 'external/post', data))
        array = img.get_fdata()
        array = np.clip(array, 0, 100)
        array = np.where(array==100, 0, array)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        print(data, img.shape)
        file_name = os.path.join('../data/clusterMask/external/post', data)
        nib.save(img, file_name)
        index = subprocess.run(['cluster', f'--in={file_name}', '--thresh=0.1', f'--oindex={file_name}', f'--minclustersize={512*512}'], capture_output=True).stdout
        index = int(index.decode().split('\n')[1].split('\t')[0])
        img = nib.load(file_name)
        array = img.get_fdata()
        array = np.where(array==index, 1, 0)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, file_name)
        subprocess.run(['fslmaths', f'{file_name}', '-fillh', f'{file_name}']) # 100 이상 voxel을 가졌던 뇌 안쪽 부분 채우기
        img = nib.load(os.path.join(dest, 'external/post', data))
        array = img.get_fdata()
        mask = nib.load(file_name)
        mask = mask.get_fdata()
        array[mask==0] = 0
        array = np.where(array<=0, 0, array) # 혹시 뇌 안쪽에 음수값이 있었을 경우에 제거.
        index = np.argwhere(array>0)
        array = array[np.min(index[:, 0]):np.max(index[:, 0]), np.min(index[:, 1]):np.max(index[:, 1]), np.min(index[:, 2]):np.max(index[:, 2])]
        print(array.shape)
        img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
        nib.save(img, os.path.join('../data/clipNClustered/external/post', data))
        array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
        array = np.transpose(array, (2, 3, 0, 1)) #  Video must be atleast 4 dimensions: time, channels, height, width
        # wandb.log({f'clipNClustered/internal/{data}': wandb.Video(array, fps=4), f'clipNClustered/internal/{data}/scaled': wandb.Video(np.clip(array, 0, 80), fps=4)}) 
        wandb.log({f'clipNClustered/external/post/{data}': wandb.Video(array, fps=4)}) 

  0%|          | 0/309 [00:00<?, ?it/s]

00037018_2.nii.gz (512, 562, 30)
(474, 475, 27)


  0%|          | 1/309 [00:03<19:50,  3.87s/it]

00038043_2.nii.gz (512, 512, 69)
(427, 511, 65)


  1%|          | 2/309 [00:15<41:40,  8.14s/it]

...

 99%|█████████▉| 307/309 [37:22<00:20, 10.35s/it]

01891080_2.nii.gz (512, 512, 70)
(492, 492, 66)


100%|█████████▉| 308/309 [37:34<00:10, 10.79s/it]

01893382_2.nii.gz (512, 512, 65)
(415, 467, 58)


100%|██████████| 309/309 [37:43<00:00,  7.32s/it]


# 위에 한것들을 함수로 만듦

In [6]:
def cut(data, mm, srcDir, destDir):
    '''
    data: name of the nifti data
    mm: mm length to cut from the top
    srcDir: path to the data source directory
    destDir: path to the output directory
    '''
    img = nib.load(os.path.join(srcDir, data))
    vs = img.header.get_zooms()
    shape = img.shape
    cut_slice = int(np.ceil(mm/vs[2]))
    new = img.get_fdata()[:, :, -min(cut_slice, shape[2]):]
    new = nib.Nifti1Image(new, img.affine, img.header)
    nib.save(new, os.path.join(destDir, data))
    
def removeBone(data, srcDir, tmpDir, destDir):
    '''
    data: name of the nifti data
    srcDir: path to the data source directory
    tmpDir: path to the clustering mask directory
    destDir: path to the output directory
    '''
    img = nib.load(os.path.join(srcDir, data))
    array = img.get_fdata()
    array = np.clip(array, 0, 100)
    array = np.where(array==100, 0, array)
    img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
    file_name = os.path.join(tmpDir, data)
    nib.save(img, file_name)
    index = subprocess.run(['cluster', f'--in={file_name}', '--thresh=0.1', f'--oindex={file_name}', f'--minclustersize={512*512}'], capture_output=True).stdout
    index = int(index.decode().split('\n')[1].split('\t')[0])
    img = nib.load(file_name)
    array = img.get_fdata()
    array = np.where(array==index, 1, 0)
    img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
    nib.save(img, file_name)
    subprocess.run(['fslmaths', f'{file_name}', '-fillh', f'{file_name}'])
    img = nib.load(os.path.join(srcDir, data))
    array = img.get_fdata()
    mask = nib.load(file_name)
    mask = mask.get_fdata()
    array[mask==0] = 0
    array = np.where(array<=0, 0, array)
    index = np.argwhere(array>0)
    array = array[np.min(index[:, 0]):np.max(index[:, 0]), np.min(index[:, 1]):np.max(index[:, 1]), np.min(index[:, 2]):np.max(index[:, 2])]
    img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
    nib.save(img, os.path.join(destDir, data))
    return array


# 본원 post data만 다시 해보기

In [ ]:
# dest = "/ssd_data1/syjeong/hematoma/data/fu_snuh/nifti"
# os.makedirs(dest, exist_ok=True)
# source = "/ssd_data1/syjeong/hematoma/data/fu_snuh/hematoma_expansion_patients_for_post_snuh"
# folder = [r"2 CT angio pefusion (pre)", r"2 hematoma", r"CT angio perfusion (pre)", r"hematoma"]
# for f in folder:
#     directory = os.path.join(source, f)
#     for p in os.listdir(directory):
#         patient = os.path.join(directory, p)
#         subprocess.run(['dcm2niix', '-z', 'y', '-b', 'n', '-x', 'y', '-f', '%f', '-o', dest, patient])
# for f in os.listdir(dest):
#     if ('Tilt' in f) or('Eq' in f) or ('ROI' in f) or ('a' in f):
#         subprocess.run(['rm', f'{os.path.join(dest, f)}'])

In [ ]:
dest = "/ssd_data1/syjeong/hematoma/data/fu_snuh/nifti"
with wandb.init(project="HE", name="brinct_final_post", config={"data":"brinct_final", "directory":"clipNClustered internal post"}) as run:
   for data in tqdm(sorted(os.listdir(dest))):
      cut(data, 200, dest, '../data/cut20cm/internal/post')
      array = removeBone(data, '../data/cut20cm/internal/post', '../data/clusterMask/internal/post', '../data/clipNClustered/internal/post')
      array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
      array = np.transpose(array, (2, 3, 0, 1)) #  Video must be atleast 4 dimensions: time, channels, height, width
      # wandb.log({f'clipNClustered/internal/{data}': wandb.Video(array, fps=4), f'clipNClustered/internal/{data}/scaled': wandb.Video(np.clip(array, 0, 80), fps=4)}) 
      wandb.log({f'clipNClustered/internal/post/{data}': wandb.Video(array, fps=4)}) 

clipNClustered 에 있는 값의 median 값으로 upper limit clip한 다음에 mri_synthstrip 돌려보기

돌려서 나온 데이터에서 양수값을 다 1로 바꿔서 mask로써 사용.

해당 mask가 있는 부분만 원래 데이터에서 남기기.

In [42]:
os.makedirs('../data/stripMask/internal/pre', exist_ok=True)
os.makedirs('../data/stripMask/internal/post', exist_ok=True)
os.makedirs('../data/stripMask/external/pre', exist_ok=True)
os.makedirs('../data/stripMask/external/post', exist_ok=True)

In [43]:
os.makedirs('../data/stripped/internal/pre', exist_ok=True)
os.makedirs('../data/stripped/internal/post', exist_ok=True)
os.makedirs('../data/stripped/external/pre', exist_ok=True)
os.makedirs('../data/stripped/external/post', exist_ok=True)

In [7]:
def removeLeather(data, srcDir, tmpDir, destDir):
    img = nib.load(os.path.join(srcDir, data))
    array = img.get_fdata()
    array = np.clip(array, 0, np.median(array[array>0]))
    img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
    file_name = os.path.join(tmpDir, data)
    nib.save(img, file_name)
    subprocess.run(['mri_synthstrip', '-i', file_name, '-o', file_name])
    mask = nib.load(file_name).get_fdata()
    img = nib.load(os.path.join(srcDir, data))
    array = img.get_fdata()
    array[mask==0] = 0
    index = np.argwhere(array>0)
    array = array[np.min(index[:, 0]):np.max(index[:, 0]), np.min(index[:, 1]):np.max(index[:, 1]), np.min(index[:, 2]):np.max(index[:, 2])]
    img = nib.Nifti1Image(array, img.affine, img.header, img.extra, img.file_map)
    nib.save(img, os.path.join(destDir, data))
    return array

In [ ]:
cp -v /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/49288696_Tilt_Eq_1.nii.gz /ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/49288696.nii.gz

In [13]:
cp -v /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/45105597.nii.gz /ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/45105597.nii.gz

'/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/45105597.nii.gz' -> '/ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/45105597.nii.gz'


In [16]:
subprocess.run(['dcm2niix', '-z', 'y', '-b', 'n', '-f', '%f', '-o', "/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final", "/ssd_data1/syjeong/hematoma/server202/data/raw/dcm/brainct/43366093"])

Chris Rorden's dcm2niiX version v1.0.20230411  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
Found 32 DICOM file(s)
Convert 32 DICOM as /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/43366093 (512x512x32x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/43366093.nii"
Conversion required 91.619034 seconds (0.149555 for core code).


CompletedProcess(args=['dcm2niix', '-z', 'y', '-b', 'n', '-f', '%f', '-o', '/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final', '/ssd_data1/syjeong/hematoma/server202/data/raw/dcm/brainct/43366093'], returncode=0)

In [17]:
cp -v /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/43366093.nii.gz /ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/43366093.nii.gz

'/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/43366093.nii.gz' -> '/ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/43366093.nii.gz'


In [19]:
cp -v /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/26001841_Tilt_Eq_1.nii.gz /ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/26001841.nii.gz

'/ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/26001841_Tilt_Eq_1.nii.gz' -> '/ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/26001841.nii.gz'


In [ ]:
cp -v /ssd_data1/syjeong/hematoma/server202/data/raw/dcm2niix/brainct_final/26001841_Tilt_Eq_1.nii.gz /ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final/26001841.nii.gz

In [20]:
# dest = "../data/cut20cm"
# source = "../data/raw/nifti"
cut('26001841.nii.gz', 200, '/ssd_data1/syjeong/hematoma/server202/data/raw/nifti/brainct_final', '../data/cut20cm/internal/pre')
array = removeBone('26001841.nii.gz', '../data/cut20cm/internal/pre', '../data/clusterMask/internal/pre', '../data/clipNClustered/internal/pre')
source = "../data/clipNClustered"
stripMaskdir = "../data/stripMask"
dest = "../data/stripped"
subdir = "internal/pre"
array = removeLeather('26001841.nii.gz', os.path.join(source, subdir), os.path.join(stripMaskdir, subdir), os.path.join(dest, subdir))

Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/pre/26001841.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/pre/26001841.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



In [51]:
source = "../data/clipNClustered"
stripMaskdir = "../data/stripMask"
dest = "../data/stripped"

subdir = "internal/pre" # 여기만 바꾸면 됨

with wandb.init(project="HE", name=f'{subdir}', config={"data":f'{subdir}', "directory":f'{dest}'}) as run:
    for data in tqdm(sorted(os.listdir(os.path.join(source, subdir)))): 
        array = nib.load(os.path.join(source, subdir, data)).get_fdata()
        print(data, np.median(array[array>0]))
        array = removeLeather(data, os.path.join(source, subdir), os.path.join(stripMaskdir, subdir), os.path.join(dest, subdir))
        array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
        array = np.transpose(array, (2, 3, 0, 1)) 
        wandb.log({f'{dest}/{subdir}/{data}': wandb.Video(array, fps=4)}) 

  0%|          | 0/568 [00:00<?, ?it/s]

07701739.nii.gz 27.01147853396833
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/pre/07701739.nii.gz
Processing frame (of 1): 1 

done
Masked image saved to: ../data/stripMask/internal/pre/07701739.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



  0%|          | 1/568 [00:04<45:40,  4.83s/it]

07751950.nii.gz 27.99519751686603
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/pre/07751950.nii.gz
Processing frame (of 1): 1 

...

100%|█████████▉| 566/568 [49:33<00:10,  5.38s/it]

71650003.nii.gz 23.979126886697486
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/pre/71650003.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/pre/71650003.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



100%|█████████▉| 567/568 [49:37<00:05,  5.01s/it]

72605291.nii.gz 34.02614872669801
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/pre/72605291.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/pre/72605291.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



100%|██████████| 568/568 [49:44<00:00,  5.25s/it]


In [53]:
subdir = "internal/post" # 여기만 바꾸면 됨

with wandb.init(project="HE", name=f'{subdir}', config={"data":f'{subdir}', "directory":f'{dest}'}) as run:
    for data in tqdm(sorted(os.listdir(os.path.join(source, subdir)))[119:]): 
        array = nib.load(os.path.join(source, subdir, data)).get_fdata()
        print(data, np.median(array[array>0]))
        array = removeLeather(data, os.path.join(source, subdir), os.path.join(stripMaskdir, subdir), os.path.join(dest, subdir))
        array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
        array = np.transpose(array, (2, 3, 0, 1)) 
        wandb.log({f'{dest}/{subdir}/{data}': wandb.Video(array, fps=4)}) 

  0%|          | 0/321 [00:00<?, ?it/s]

31995784.nii.gz 29.977402259595692
Configuring model on the CPU
Running SynthStrip model version 1


Input image read from: ../data/stripMask/internal/post/31995784.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/post/31995784.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



  0%|          | 1/321 [00:08<44:31,  8.35s/it]

32115031.nii.gz 38.00417831353843
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/post/32115031.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/post/32115031.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



...

 99%|█████████▉| 319/321 [54:59<00:13,  6.55s/it]

71650003.nii.gz 38.99539530649781
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/post/71650003.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/post/71650003.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



100%|█████████▉| 320/321 [55:03<00:05,  5.87s/it]

72605291.nii.gz 29.98677710769698
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: ../data/stripMask/internal/post/72605291.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: ../data/stripMask/internal/post/72605291.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



100%|██████████| 321/321 [55:10<00:00, 10.31s/it]


In [ ]:
# subdir = "external/pre" # 여기만 바꾸면 됨

# with wandb.init(project="HE", name=f'{subdir}', config={"data":f'{subdir}', "directory":f'{dest}'}) as run:
#     for data in tqdm(sorted(os.listdir(os.path.join(source, subdir)))): 
#         array = nib.load(os.path.join(source, subdir, data)).get_fdata()
#         print(data, np.median(array[array>0]))
#         array = removeLeather(data, os.path.join(source, subdir), os.path.join(stripMaskdir, subdir), os.path.join(dest, subdir))
#         array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
#         array = np.transpose(array, (2, 3, 0, 1)) 
#         wandb.log({f'{dest}/{subdir}/{data}': wandb.Video(array, fps=4)}) 

In [ ]:
# subdir = "external/post" # 여기만 바꾸면 됨

# with wandb.init(project="HE", name=f'{subdir}', config={"data":f'{subdir}', "directory":f'{dest}'}) as run:
#     for data in tqdm(sorted(os.listdir(os.path.join(source, subdir)))): 
#         array = nib.load(os.path.join(source, subdir, data)).get_fdata()
#         print(data, np.median(array[array>0]))
#         array = removeLeather(data, os.path.join(source, subdir), os.path.join(stripMaskdir, subdir), os.path.join(dest, subdir))
#         array = np.repeat(array[...,np.newaxis], 3, -1)  # H W T 3
#         array = np.transpose(array, (2, 3, 0, 1)) 
#         wandb.log({f'{dest}/{subdir}/{data}': wandb.Video(array, fps=4)}) 

# 본원 pre data랑 post data 갯수가 안맞는다

pre에 post에 없는 data가 128개 있다.

post에 있는 data는 pre에 다 있다

internal pre data 568개

internal post data 440개